There are native and numpy variants for some segments. Each segment expects a native input but may output numpy data structures.

### Basic Imports

In [ ]:
import os
import sys
import argparse

In [ ]:
import numpy as np
import copy

### Command Line Arguments

In [ ]:
###### Command Line Emulation
sys.argv = ["assignment_2.py", 
            "--input=sensors_graz.csv", 
            "--output=assignment_2.csv",
            "--resampling_delta", "3600",
            "--window_smoothing", "2592000"]
sys.argv

In [ ]:
# parse args 
arg_parser = argparse.ArgumentParser(description="Load and clean a sensor data file")
arg_parser.add_argument("-i", "--input", type=str, required=True, help="Path to the input file")
arg_parser.add_argument("-o", "--output", type=str, required=True, help="Path to the output file")
arg_parser.add_argument("-s", "--sensors", type=int, nargs="+", default=[], help="Sensor IDs to process")
arg_parser.add_argument("-r", "--resampling_delta", type=int, default=300, help="The timedelta for the resampling")
arg_parser.add_argument("-w", "--window_smoothing", type=int, help="The timedelta for the smoothing")

cmd_args = arg_parser.parse_args()

### Basic Checks

In [ ]:
# Check if input file exists
if not os.path.isfile(cmd_args.input):
    raise FileNotFoundError("Input file does not exist")
    
if os.path.splitext(cmd_args.input)[1] != ".csv":
    raise NameError("Input file has to be a CSV (.csv ending)")

# Check if output directory exists
output_dir, output_file = os.path.split(cmd_args.output)
if not os.path.isdir(output_dir) and output_dir != "":
    raise NotADirectoryError("Output directory does not exist")

### Reading and Parsing the Input File

In [ ]:
# Helper function to create data entries from lines

def create_data_entry(headers, data):
    new_entry = dict()
    for key, value in zip(headers, data):
        new_entry[key] = value
    return new_entry

In [ ]:
# read file 

with open(cmd_args.input, "r") as input_file:
    # put all lines into a single list
    lines = [l.strip() for l in input_file.readlines()]
    # the first line contains the headers. so we cut that off
    headers = lines[0].split(",")
    # the data is anything below the header line
    data_lines = lines[1:]

We store the data in a dictionary. This is constructed as follows: The dict has one entry for each sensor (`sensor_id`) and the values contain the sensor id (again), location information (lat/lon) which type the sensor is and a dictionary for the data. In that data we have a list for timestamps and lists containing the values for each metric the sensor reports

In [ ]:
sensors = dict()

# read through each line
for data_line in data_lines:
    data_line = data_line.split(",")
    entry = create_data_entry(headers, data_line)
    sensor_id = int(entry["sensor_id"])
    timestamp = np.datetime64(entry['timestamp']) # convert this to a datetime object
    p_1 = entry['P1']
    p_2 = entry['P2']
        
    # check if the this is a sensor we want
    if cmd_args.sensors == [] or sensor_id in cmd_args.sensors:
        # If we do not have the sensor in the dict already, then create a new entry with the basic structure
        if sensor_id not in sensors:
            sensors[sensor_id] = dict()
            sensors[sensor_id]['sensor_id'] = sensor_id
            sensors[sensor_id]['lat'] = entry['lat']
            sensors[sensor_id]['lon'] = entry['lon']
            sensors[sensor_id]['sensor_type'] = entry['sensor_type']
            sensors[sensor_id]['data'] = dict()
            sensors[sensor_id]['data']['timestamp'] = []
            sensors[sensor_id]['data']['P1'] = []
            sensors[sensor_id]['data']['P2'] = []
        
        # here we can be sure the entry exists so we can just add the data
        sensors[sensor_id]['data']['timestamp'].append(np.datetime64(timestamp)) # cast to a numpy 
        sensors[sensor_id]['data']['P1'].append(float(p_1)) # cast to a float
        sensors[sensor_id]['data']['P2'].append(float(p_2)) # cast to a float

In [ ]:
# Let's do a quick check to see if the data lists are all the same length. If not something strange went wrong...

for sensor_id in sensors:
    data_dict = sensors[sensor_id]['data']
    
    if len(data_dict['timestamp']) != len(data_dict['P1']) and len(data_dict['timestamp']) != len(data_dict['P2']):
        raise ValueError("Something went wrong. The data lengths do not match")

### Sort the data entries (by timestamp)

#### (i) Pure python variant

In [ ]:
for sensor_id in sensors:
    data_dict = sensors[sensor_id]['data']
    
    value_indices = list(range(len(data_dict['timestamp'])))
    sort_order = sorted(value_indices, key=data_dict['timestamp'].__getitem__)
    # variant with a lambda. does exactly the same actually just doesnt use a hidden built in function
    # sort_order = sorted(value_indices, key=lambda x: data_dict['timestamp'][x])
    
    # now let's rearrange the timestamps and value lists
    for key in ["timestamp", "P1", "P2"]:
        sensors[sensor_id]['data'][key] = [data_dict[key][x] for x in sort_order]

#### Numpy variant

### Smooth Values

In [ ]:
# copy the original values to a "data_orig" entry in the sensors dict.
# Use a deepcopy as the objects in the data dict are mutable! Thus we use the copy package!

for sensor_id in sensors:
    sensors[sensor_id]['data_orig'] = copy.deepcopy(sensors[sensor_id]['data'])

#### Cut single outliers

Outlier definition: A value is higher than twice the mean of the previous and next value. (> prev+next) <br />
When we encounter an outlier we interpolate between the previous and the next. (Do this online)

In [ ]:
def is_outlier(val, prev_val, next_val):
    if val is None or prev_val is None or next_val is None:
        return False
    return val > (prev_val + next_val)

In [ ]:
num_smoothing_passes = 5
for sensor_id in sensors:
    for _ in range(num_smoothing_passes):
        for key in ["P1", "P2"]:
            data_list = sensors[sensor_id]['data'][key]
            for current_index in range(1, len(data_list) - 1):
                if is_outlier(data_list[current_index], data_list[current_index - 1], data_list[current_index + 1]):
                    data_list[current_index] = (data_list[current_index - 1] + data_list[current_index + 1]) / 2

#### Resampling to the interval

In [ ]:
# we use the median as smoothing function as it is less affected by outliers. alternative would be the mean

def resampling_function(array):
    """
        calculates the median of list of integers
    """
    # quick check to see if all elements in the list are numeric (float or int)
    if sum(isinstance(x, (int, float)) for x in array) != len(array):
        print(array[0], type(array[0]))
        raise TypeError("Expected a list of numerical values. Got at least one non-numerical")
    
    sorted_array = sorted(array)
    num_values = len(array)
    median_index = (num_values - 1) // 2
    median = None

    if (num_values % 2):
        median = sorted_array[median_index]
    else:
        median = (sorted_array[median_index] + sorted_array[median_index + 1])/2
    return median
    
# numpy variant
# resampling_function = np.mean

In [ ]:
# resample the values. calculate the resampling function per window. use timestamp and timedelta datatypes. 
# start at 00:00 on the first day and end at 24:00 on the last. 
# center the resampling function around the current interval
# source the data from the data_orig object and store it into data.

# 30 minutes = 30 * 60 seconds

resampling_delta = np.timedelta64(cmd_args.resampling_delta, "s")

for sensor_id in sensors:
    data_dict = sensors[sensor_id]['data_orig']
    current_timestamp = np.datetime64(np.datetime64(data_dict['timestamp'][0], "D"), "s")
    last_timestamp = np.datetime64(np.datetime64(data_dict['timestamp'][-1], "D") + 1, "s")
    
    # create a new (empty) data dict. we put the smoothed values in there
    # and then overwrite the entire data object at the end
    data_dict_new = {"timestamp": [],
                     "P1": [], 
                     "P2": []}
    
    # since we have sorted the data by timestamp we can carry an index indicating where we are in the list.
    # this stops us from always having to iterate over the entire list
    current_index = 0
    
    # do as long as we have not reached the ending timestamp
    while current_timestamp <= last_timestamp:
        # the indices of the values within the current windown. we want the indices 
        # to smooth the values for both data lists (P1 and P2)!
        value_indices = []
        # do forever (or until we reach the end). we manually break out of the loop
        while current_index < len(data_dict['timestamp']):
            # break out of the loop when the timestamp of the current index is larger than 
            # the current timestamp plus half the resampling delta (since we center around them)
            if data_dict['timestamp'][current_index] >= current_timestamp + (resampling_delta / 2):
                break
            # Add the index to the window
            value_indices.append(current_index)
            
            current_index += 1
        
        # now we calculate the smoothed values using the smoothing function and add this to the new data object
        data_dict_new['timestamp'].append(current_timestamp)
        for key in ["P1", "P2"]:
            # the window could be empty (sensor could have been offline). in that case just add "None"
            if len(value_indices) == 0:
                 data_dict_new[key].append(None)
            else:
                data_dict_new[key].append(resampling_function([data_dict[key][x] for x in value_indices]))
            
        #now that we have finished this timestamp move on to the next
        current_timestamp += resampling_delta
        

    # overwrite the data object (do not touch the data_orig)
    sensors[sensor_id]['data'] = data_dict_new

#### Rolling Means
This smoothes the whole data over a longer duration

In [ ]:
# this variant takes edges into account

if cmd_args.window_smoothing is not None:
    window_size = int((np.timedelta64(cmd_args.window_smoothing, "s") / 2) / resampling_delta)
    
    for sensor_id in sensors:
        for key in ["P1", "P2"]:
            data_list = sensors[sensor_id]['data'][key]
            data_list_new = copy.deepcopy(data_list)
            
            for current_index in range(len(data_list)):
                i_from = max(current_index-window_size, 0)
                i_to = min(current_index+window_size+1, len(data_list))
                valid_list = [x for x in data_list[i_from:i_to] if x is not None]
                data_list_new[current_index] = sum(valid_list) / max(len(valid_list), 1)
                
            sensors[sensor_id]['data'][key] = data_list_new

### Plotting
This functionality below is provided with the assignment!

In [ ]:
from ass_2_plot import plot_data

In [ ]:
for sensor_id, sensor_data in sensors.items():
    plot_data("Sensor ID: {id}".format(id=sensor_id), 
              sensor_data['data_orig'], 
              sensor_data['data'], 
              filename="{outname}_{id}.png".format(outname=os.path.basename(cmd_args.output), id=sensor_id))

In [ ]:
if "location" in headers:
    headers.remove("location")

In [ ]:
dump_lines = []
for sensor_id, sensor in sensors.items():
    data = sensor['data']
    for ts, p1, p2 in zip(data['timestamp'], data['P1'], data['P2']):
        ts = str(ts).replace("T", " ")
        if p1 == None:
            p1 = "NaN"
        else:
            p1 = np.round(p1, 3)
        if p2 == None:
            p2 = "NaN"
        else:
            p2 = np.round(p2, 3)
        line = [sensor_id, sensor['sensor_type'], sensor['lat'], sensor['lon'], ts, p1, p2]
        dump_lines.append(line)

dump_lines.sort(key=lambda x: (x[4], x[0]))
dump_lines = [','.join((str(y) for y in x)) for x in dump_lines]

In [ ]:
with open(cmd_args.output, "w") as output_file:
    output_file.write(",".join(headers))
    output_file.write("\n")
    for line in dump_lines:
        output_file.write(line)
        output_file.write("\n")